In [12]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from datetime import datetime
import time

In [13]:
options = Options()
options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)
wait = WebDriverWait(driver, 3)
url = "https://www.idealo.de/preisvergleich/OffersOfProduct/4776635_-dc-500-e-bernardo.html"
driver.get(url)
time.sleep(3)  # wichtig wegen JS-Ladezeit

In [14]:
def reject_cookies_if_present(driver, timeout=5):
    try:
        wait = WebDriverWait(driver, timeout)
        time.sleep(1)
        
        # JavaScript to find and click the deny button inside shadow DOM
        script = """
        const shadowHost = document.querySelector('#usercentrics-cmp-ui');
        if (shadowHost && shadowHost.shadowRoot) {
            const denyButton = shadowHost.shadowRoot.querySelector('#deny');
            if (denyButton) {
                denyButton.click();
                return true;
            }
        }
        return false;
        """
        
        result = driver.execute_script(script)
        if result:
            print("Successfully clicked deny button in shadow DOM")
            time.sleep(1)
            return True
        else:
            print("Could not find deny button in shadow DOM")
            return False
    except Exception as e:
        print(f"Error: {e}")
        return False

reject_cookies_if_present(driver)

Successfully clicked deny button in shadow DOM


True

In [ ]:
""" listprice element """
price_element = wait.until(
    EC.presence_of_element_located(
        (By.XPATH, "//*[@class='productOffers-listItemOfferPrice']")
    )
)


In [ ]:
""" standing out price element
price_element = wait.until(
    EC.presence_of_element_located(
        (By.XPATH, "//*[@id='oopStage-conditionButton-new']/div/div[1]/div[2]/strong")
    )
)
""""""


TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x0000000100a4bdfc cxxbridge1$str$ptr + 3031016
1   chromedriver                        0x0000000100a43cb8 cxxbridge1$str$ptr + 2997924
2   chromedriver                        0x000000010053eb90 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 74192
3   chromedriver                        0x0000000100585ab4 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 364788
4   chromedriver                        0x00000001005c6a28 _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 630888
5   chromedriver                        0x000000010057a22c _RNvCsgXDX2mvAJAg_7___rustc35___rust_no_alloc_shim_is_unstable_v2 + 317548
6   chromedriver                        0x0000000100a10194 cxxbridge1$str$ptr + 2786176
7   chromedriver                        0x0000000100a13900 cxxbridge1$str$ptr + 2800364
8   chromedriver                        0x00000001009f042c cxxbridge1$str

In [18]:
price_text = price_element.text.strip()

def parse_price(price_str):
    # Remove the € symbol and whitespace
    price_str = price_str.replace('€', '').strip()
    
    # Replace German number format: 3.890,00 → 3890.00
    # Remove thousands separator (.) and replace decimal separator (,) with (.)
    price_str = price_str.replace('.', '').replace(',', '.')
    
    # Convert to float, then to int if no decimals needed
    price_float = float(price_str)
    price_int = int(price_float)
    
    return price_int

price_number = parse_price(price_text)
print(f"Price number: {price_number}")


Price number: 459
